In [1]:
from tkinter import *
from PIL import Image, ImageTk
from tkinter import filedialog, Frame, ttk
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import glob

Cargamos el Dataset ya creado para entrenar un modelo

In [2]:
pickle_in = open('X.pickle','rb')
X = pickle.load(pickle_in)
pickle_in = open('y.pickle','rb')
y = pickle.load(pickle_in)
pickle_in = open('Z.pickle','rb')
Z = pickle.load(pickle_in)

In [3]:
# Divido entre datos de entrenamiento y de prueba
X_train, X_test, y_train, y_test, Z_train, Z_test = train_test_split(X, y, Z, stratify=y, random_state=0)

In [4]:
#Entrenamos con knn
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=1)

Definimos funciones

In [5]:
#Dado un número nos dice cual es el tipo de sello al que está asociado ese número
#Esto lo queremos pues knn nos devuelve números y nosotros los expresamos con el tipo de sello
def clase(i):
    n=''
    if (i==0):
        n='Dado'
    if (i==1):
        n='Dardo'
    if (i==2):
        n='Foco'
    if (i==3):
        n='Maceta'
    if (i==4):
        n='Oso'
    if (i==5):
        n='Pulpo'
    if (i==6):
        n='Rueda'
    return n

In [6]:
#Dada una imágen predice el tipo de sello que ocupa y lo devuelve como string
def predecir(imagen):
    img = cv2.imread(imagen, cv2.IMREAD_GRAYSCALE)
    img_N = cv2.resize(img,(140,140))
    img_V = (np.asarray(img_N)).flatten()
    #prediccion = knn.predict([img_V])
    #for i in range(7):
    #    if (prediccion==i):
    #        p=clase(i)
    p=clase(knn.predict([img_V]))
    return(p)                

In [7]:
#Dada una imágen la recorta en imágenes más pequeñas y las guarda en la carpeta usuario
def recortar(imagen):
    for im in glob.glob("Usuario/*.jpg"):
        try:
            os.remove(im)
        except OSError as e:
            print(f"Error:{ e.strerror}")   
    image = imagen
    gradX = cv2.Sobel(image, cv2.CV_32F, dx=1, dy=0, ksize=-1)
    gradY = cv2.Sobel(image, cv2.CV_32F, dx=0, dy=1, ksize=-1)
    gradient = cv2.subtract(gradX, gradY)
    gradient = cv2.convertScaleAbs(gradient)
    blurred = cv2.blur(gradient, (9, 9))
    _, thresh = cv2.threshold(blurred, 90, 255, cv2.THRESH_BINARY)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (25, 25))
    closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    closed = cv2.erode(closed, None, iterations=4)
    closed = cv2.dilate(closed, None, iterations=4)
    a=100
    c=0
    Imagen=closed
    while a>2:
        (cntsf, xf) = cv2.findContours(Imagen.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cf = sorted(cntsf, key=cv2.contourArea, reverse=True)[0]
        rectf = cv2.minAreaRect(cf)
        boxf = np.int0(cv2.boxPoints(rectf))
        Xsf = [i[0] for i in boxf]
        Ysf = [i[1] for i in boxf]
        x1f = min(Xsf)
        x2f = max(Xsf)
        y1f = min(Ysf)
        y2f = max(Ysf)
        cropImgf= image[y1f:y2f, x1f:x2f]
        A=str(c)
        cv2.imwrite("Usuario/imagen"+A+".jpg", cropImgf)
        b=0
        dimensionesf = (closed.shape[0],closed.shape[1])
        arrayf=np.zeros(dimensionesf)
        for i in range(closed.shape[1]):
            for j in range(closed.shape[0]):
                if (x1f < i < x2f and y1f < j < y2f):
                    arrayf[j,i]=0
                else:
                    arrayf[j,i]=Imagen[j,i]
        for i in range(closed.shape[1]):
            for j in range(closed.shape[0]):
                if (arrayf[j,i]>0):
                    b=b+1
        a=(b*100)/(closed.shape[0]*closed.shape[1])
        c=c+1
        Imagen= np.uint8(arrayf)

In [8]:
def info(i):
    n=''
    if (i=='Dado'):
        n='Los Dados tienen 6 caras'
    if (i=='Dardo'):
        n='El Dardo'
    if (i=='Foco'):
        n='Foco 3'
    if (i=='Maceta'):
        n='Maceta 4'
    if (i=='Oso'):
        n='Oso 5'
    if (i=='Pulpo'):
        n='Pulpo 6'
    if (i=='Rueda'):
        n='Rueda 7'
    return n

Interfaz gráfica

In [45]:
class Clasificador(Frame):
    def __init__(self,master=None):
        super().__init__(master)#,width=2000,height=500,bg='#F1F8EC')
        self.master=master
        self.place(relwidth=1,relheight=1)
        self.pedir_imagen()
    
    def pedir_imagen(self):
        fpedir=Frame(self, height=100, bg='cyan')
        fpedir.pack(fill='x')#(relwidth=1)#pack(expand=True)#grid(row=0,column=0,columnspan=2)
        btn_seleccionar_imagen = Button(fpedir, text='Seleccionar imagen...', command=self.seleccionar_imagen)
        btn_seleccionar_imagen.pack(pady=15)#grid(row=0, column=0,columnspan=2, pady=15,)

        
    
        
        
        
    def seleccionar_imagen(self):
        pass
        archivo = filedialog.askopenfilename(filetypes=(('Archivos de imagen', '*.png'),("jpeg files","*.jpg")))
        #fcontenido.destroy()
        fcontenido = Frame(self,bg='yellow')
        fcontenido.pack(fill='both',expand=1)  
        
        if archivo is not None:
            
            
            
            fderecha=Frame(fcontenido,bg='blue')
            fderecha.pack(side='right',fill='both',expand=1)##.pack(fill='y',side='right')place(relwidth=.5,y=50)

            fimagen=Frame(fcontenido,bg='red')#width='200', height='600',bg='red')
            fimagen.pack(side='left',fill='both',expand=1)##pack(fil='y',side='left')place(relwidth=.5,relheight=1,y=50)

            my_canvas=Canvas(fderecha,bg='green')
            my_canvas.pack(side='left',fill='both',expand=1)

            my_scrollbar=ttk.Scrollbar(fderecha,orient='vertical',command=my_canvas.yview)
            my_scrollbar.pack(side='right',fill='y')

            my_canvas.configure(yscrollcommand=my_scrollbar.set)
            my_canvas.bind('<Configure>', lambda e: my_canvas.configure(scrollregion=my_canvas.bbox('all')))

            second_frame=Frame(my_canvas,bg='green')
            second_frame.pack(side='right',fill='both',expand=1)

            my_canvas.create_window((0,0),window=second_frame, anchor='nw')

            lbl_im=Label(fimagen,text='Aquí aparecerá tu imágen')
            lbl_im.pack()
            #for widget in fderecha.winfo_children():
            #    widget.destroy()
            
            #for widget in fimagen.winfo_children():
            #    widget.destroy()
            
            imagen = Image.open(archivo)

            imagen = imagen.resize((200, 200), Image.ANTIALIAS)

            imagen = ImageTk.PhotoImage(imagen)
            
            
            lbl_textimg=Label(fimagen,text='La imágen que seleccionaste:')
            lbl_textimg.pack(pady=10)#.grid(row=0,column=0,pady=15,padx=15)
            
            lbl_imagen = Label(fimagen, image=imagen)
            lbl_imagen.image = imagen
            lbl_imagen.pack(pady=10)#.grid(row=1, column=0,columnspan=2,pady=10, padx=10)
            
            
            

        if archivo is not None:
            img = cv2.imread(archivo, cv2.IMREAD_GRAYSCALE)
            recortar(img)
            Numero=len(glob.glob("Usuario/*.jpg"))
            for i in range(Numero):
                exec('imagen_{} = Image.open("Usuario/imagen"+str(i)+".jpg")'.format(i))
                exec('imagen_{} = imagen_{}.resize((60, 60), Image.ANTIALIAS)'.format(i,i))
                exec('imagen_{} = ImageTk.PhotoImage(imagen_{})'.format(i,i))
                
                #fresultado.insert(END,imagen_{},"La imágen es del tipo",)
                #fresultado.insert(END,lbl_imagen_{},lbl_tipo_{})
                

                exec('lbl_imagen_{} = Label(second_frame, image=imagen_{})'.format(i,i))
                exec('lbl_imagen_{}.image = imagen_{}'.format(i,i))
                exec('lbl_imagen_{}.grid(row=i+3, column=0,pady=10, padx=10)'.format(i))
                
                #exec('img_{}=predecir("Usuario/imagen"+str(i)+".jpg")'.format(i))
                exec('lbl_tipo_{}=Label(second_frame,text="La imágen es del tipo:")'.format(i))
                exec('lbl_tipo_{}.grid(row=i+3,column=1,sticky=E)'.format(i))
                exec('lbl_img_{}=Label(second_frame,text=predecir("Usuario/imagen"+str(i)+".jpg"),width=10)'.format(i))
                exec('lbl_img_{}.config(bg="white")'.format(i))
                exec('lbl_img_{}.grid(row=i+3,column=2,pady=15, padx=10,sticky=W)'.format(i))
                #exec('inf_{}=info({})'.format(i,i))
                exec('lbl_inf_{}=Label(second_frame,text=info(predecir("Usuario/imagen"+str(i)+".jpg")))'.format(i))
                exec('lbl_inf_{}.grid(row=i+3,column=3)'.format(i))
root=Tk()
root.wm_title('Clasificador de imágenes')
root.geometry('1000x500')
app=Clasificador(root)
app.mainloop()

In [10]:
#Ejemplos de interface gráfica, solo para ayudar

In [41]:
class GUI(Frame):
    def __init__(self,master=None):
        super().__init__(master)
        self.master=master
        self.place(relwidth=1,relheight=1)
        self.boton()
        
    def boton(self):
        fboton=Frame(self,height=100, bg='cyan')
        fboton.pack(fill='x')#place(relwidth=1)
        btn_seleccionar_imagen = Button(fboton, text='Seleccionar imagen...', command=self.listadecosas)
        btn_seleccionar_imagen.pack(pady=15)
        fcontenido=Canvas(self,bg='yellow')
        fcontenido.pack(fill='both',expand=1)
        #l_prueba=Canvas(fcontenido,text='Prueba').pack()
        
    def listadecosas(self):
        fizquierda=Frame(self,bg='red')
        fizquierda.place(relwidth=.5,x=0,relheight=.8,y=50)
        
        fderecha=Frame(self,bg='green')
        fderecha.place(relwidth=.5,relx=.5,relheight=.8,y=50)
        my_canvas=Canvas(fderecha)
        my_canvas.pack(side='left',fill='both',expand=1)
        my_scrollbar=ttk.Scrollbar(fderecha,orient='vertical',command=my_canvas.yview)
        my_scrollbar.pack(side='right',fill='y')
        
        my_canvas.configure(yscrollcommand=my_scrollbar.set)
        my_canvas.bind('<Configure>', lambda e: my_canvas.configure(scrollregion=my_canvas.bbox('all')))
        
        second_frame=Frame(my_canvas)
        
        my_canvas.create_window((0,0),window=second_frame, anchor='nw')
        for i in range(100):
            exec('lbl_{}=Label(second_frame,text=str(i))'.format(i))
            exec('lbl_{}.grid()'.format(i))

root=Tk()
root.wm_title('Clasificador de imágenes')
root.geometry('1000x500')
app=GUI(root)
app.mainloop()

TclError: cannot use geometry manager pack inside .!gui which already has slaves managed by grid

In [12]:
Ventana=Tk()
Ventana.title('Clasificador de imágenes')
Ventana.geometry('1300x800')
Ventana.resizable(width=True, height=True)
fpedirimagen=Frame(Ventana,width='800', height='100',bg='blue')
fimagen=Frame(Ventana,width='200', height='600',bg='red')
fresultados=Frame(Ventana,width='700', height='600',bg='yellow')

fpedirimagen.pack()
fimagen.pack(side='left')
fresultados.pack(side='right')
Ventana.mainloop()

In [51]:
        
    def seleccionar_imagen():
        pass
        archivo = filedialog.askopenfilename(filetypes=(('Archivos de imagen', '*.png'),("jpeg files","*.jpg")))
    
        if archivo is not None:
            for widget in second_frame.winfo_children():
                widget.destroy()
            
            for widget in fimagen.winfo_children():
                widget.destroy()
            #for widget in fcontenido.winfo_children():
            #    widget.destroy()

            #for widget in fderecha.winfo_children():
            #    widget.destroy()
            
            #for widget in fimagen.winfo_children():
            #    widget.destroy()
            
            imagen = Image.open(archivo)

            imagen = imagen.resize((200, 200), Image.ANTIALIAS)

            imagen = ImageTk.PhotoImage(imagen)
            
            
            lbl_textimg=Label(fimagen,text='La imágen que seleccionaste:')
            lbl_textimg.pack(pady=10)#.grid(row=0,column=0,pady=15,padx=15)
            
            lbl_imagen = Label(fimagen, image=imagen)
            lbl_imagen.image = imagen
            lbl_imagen.pack(pady=10)#.grid(row=1, column=0,columnspan=2,pady=10, padx=10)
            
            
            

        if archivo is not None:
            img = cv2.imread(archivo, cv2.IMREAD_GRAYSCALE)
            recortar(img)
            Numero=len(glob.glob("Usuario/*.jpg"))
            for i in range(Numero):
                exec('imagen_{} = Image.open("Usuario/imagen"+str(i)+".jpg")'.format(i))
                exec('imagen_{} = imagen_{}.resize((60, 60), Image.ANTIALIAS)'.format(i,i))
                exec('imagen_{} = ImageTk.PhotoImage(imagen_{})'.format(i,i))
                
                #fresultado.insert(END,imagen_{},"La imágen es del tipo",)
                #fresultado.insert(END,lbl_imagen_{},lbl_tipo_{})
                

                exec('lbl_imagen_{} = Label(second_frame, image=imagen_{})'.format(i,i))
                exec('lbl_imagen_{}.image = imagen_{}'.format(i,i))
                exec('lbl_imagen_{}.grid(row=i+3, column=0,pady=10, padx=10)'.format(i))
                
                #exec('img_{}=predecir("Usuario/imagen"+str(i)+".jpg")'.format(i))
                exec('lbl_tipo_{}=Label(second_frame,text="La imágen es del tipo:")'.format(i))
                exec('lbl_tipo_{}.grid(row=i+3,column=1,sticky=E)'.format(i))
                exec('lbl_img_{}=Label(second_frame,text=predecir("Usuario/imagen"+str(i)+".jpg"),width=10)'.format(i))
                exec('lbl_img_{}.config(bg="white")'.format(i))
                exec('lbl_img_{}.grid(row=i+3,column=2,pady=15, padx=10,sticky=W)'.format(i))
                #exec('inf_{}=info({})'.format(i,i))
                exec('lbl_inf_{}=Label(second_frame,text=info(predecir("Usuario/imagen"+str(i)+".jpg")))'.format(i))
                exec('lbl_inf_{}.grid(row=i+3,column=3)'.format(i))
root=Tk()
root.wm_title('Clasificador de imágenes')
root.geometry('1000x500')

fpedir=Frame(root, height=100, bg='cyan')
fpedir.pack(fill='x')#(relwidth=1)#pack(expand=True)#grid(row=0,column=0,columnspan=2)
btn_seleccionar_imagen = Button(fpedir, text='Seleccionar imagen...', command=seleccionar_imagen)
btn_seleccionar_imagen.pack(pady=15)#grid(row=0, column=0,columnspan=2, pady=15,)
        
fcontenido = Frame(root,bg='yellow')
fcontenido.pack(fill='both',expand=1)

fderecha=Frame(fcontenido,bg='blue')
fderecha.pack(side='right',fill='both',expand=1)##.pack(fill='y',side='right')place(relwidth=.5,y=50)

fimagen=Frame(fcontenido,bg='red')#width='200', height='600',bg='red')
fimagen.pack(side='left',fill='both',expand=1)##pack(fil='y',side='left')place(relwidth=.5,relheight=1,y=50)
my_canvas=Canvas(fderecha,bg='green')
my_canvas.pack(side='left',fill='both',expand=1)

my_scrollbar=ttk.Scrollbar(fderecha,orient='vertical',command=my_canvas.yview)
my_scrollbar.pack(side='right',fill='y')

my_canvas.configure(yscrollcommand=my_scrollbar.set)
my_canvas.bind('<Configure>', lambda e: my_canvas.configure(scrollregion=my_canvas.bbox('all')))

second_frame=Frame(my_canvas,bg='green')
second_frame.pack(side='right',fill='both',expand=1)

my_canvas.create_window((0,0),window=second_frame, anchor='nw')




#app=Clasificador(root)
root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\David\anaconda3\lib\site-packages\PIL\Image.py", line 2916, in open
    fp.seek(0)
AttributeError: 'str' object has no attribute 'seek'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\David\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "<ipython-input-51-f791aae6dbd9>", line 21, in seleccionar_imagen
    imagen = Image.open(archivo)
  File "C:\Users\David\anaconda3\lib\site-packages\PIL\Image.py", line 2918, in open
    fp = io.BytesIO(fp.read())
AttributeError: 'str' object has no attribute 'read'
